In [1]:
# import relevant libraries

import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

In [2]:
# load all csv files with headers, except ftp-write, which does not have column names
df_back = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back.csv")
df_buffer_overflow = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_BufferOverflow.csv")
df_guess_password = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_GuessPassword.csv")
df_neptune = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_Neptune.csv")
df_nmap = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_NMap.csv")
df_normal = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_Normal.csv")
df_port_sweep = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_PortSweep.csv")
df_rootkit = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_RootKit.csv")
df_satan = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_Satan.csv")
df_smurf = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_Smurf.csv")

In [3]:
# load ftp-write without headers
df_ftp_write = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_FTPWrite.csv", header = None)